### Kinetic Parameters

This notebook contains kinetic parameters and utility functions for the final project.

Use this code with:
```python
import import_ipynb
from kinetic_parameters import rate
```